In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

In [ ]:
%pip install qdrant-client

In [3]:
client = QdrantClient('localhost', port=6333)

In [5]:
client.create_collection(
    collection_name='hello_world_collection',
    vectors_config=models.VectorPrograms(size=4, distance=models.Distance.COSINE))

AttributeError: module 'qdrant_client.http.models.models' has no attribute 'VectorPrograms'